# Chemical Concentrations at River Mouths
Longitudinal concentrations of various chemicals at a range of depths, for all locations where (large) rivers meet the ocean.  

Note that this is a parameterized widget; the specification passed to the API will not be renderable without the geostore identifier being inserted.  

_Author: Peter Kerins_  
_Created: 4 Aug 2021_  
_Environment: jupyterlab_  

## Style
- Vega chart
- Ocean Watch country page - parameterized chart
- Time series
- Line or multiline

## Data
See [explanatory notebook](../../datasets/ocn_020d_chemical_concentration_rivermouths/river-mouths_process-overview.ipynb)
 
Carto: [ocn_020d](https://resourcewatch.carto.com/u/rw-nrt/dataset/ocn_020d_chemical_concentration_rivermouths)

## Preparation

In [1]:
import json
from vega import Vega
from IPython.display import display

In [2]:
def Vega(spec):
    bundle = {}
    bundle['application/vnd.vega.v5+json'] = spec
    display(bundle, raw=True)

In [3]:
widget_width = 400
widget_height = 300

# Widget

## Demo Query
`hybas_id_5 = <basin>` used as stand-in for parameterization

```sql
SELECT hyriv_id, hybas_id_5 AS hybas_id, ord_flow, variable, dt AS date, AVG(value) AS avg_value, osm_name AS name
FROM ocn_020d_chemical_concentration_rivermouths 
WHERE hybas_id_5 = '1050030460' 
GROUP BY hyriv_id, hybas_id_5, ord_flow, variable, dt
ORDER BY hyriv_id ASC, variable ASC, dt ASC
```

## Parameterized Query
```sql
SELECT hyriv_id, hybas_id_5 AS hybas_id, osm_name AS name, ord_flow, variable, dt AS date, AVG(value) AS avg_value
FROM ocn_020d_chemical_concentration_rivermouths data
LEFT OUTER JOIN wat_068_rw0_watersheds_edit AS basins ON basins.hybas_id::VARCHAR = data.hybas_id_5 
WHERE basins.{{geostore_env}}='{{geostore_id}}' 
AND basins.level=5
AND basins.coast=1
GROUP BY hyriv_id, hybas_id_5, osm_name, ord_flow, variable, dt
ORDER BY hyriv_id ASC, variable ASC, dt ASC
```

## Specification
Vega code presumes RW-style `config` element present

In [9]:
spec=json.loads("""
{
  "schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "Chemical concentration of various nutrient-type molecules at river mouths in corresponding basin",
  "width": 400,
  "height": 250,
  "padding": 0,
  "autosize": {
    "type": "fit",
    "contains": "padding"
  },
  "signals": [
    {
      "name": "variable",
      "description": "Chemical/product of interest",
      "value": "no3"
    },
    {
      "name": "active_id",
      "value": "",
      "on": [
        {
          "events": "*:mouseover",
          "update": "datum.hyriv_id"
        }
      ]
    }
  ],
  "data": [
    {
      "name": "table",
      "url": "https://rw-nrt.carto.com/api/v2/sql?q=SELECT hyriv_id, hybas_id_5 AS hybas_id, osm_name AS name, ord_flow, variable, dt AS date, AVG(value) AS avg_value FROM ocn_020d_chemical_concentration_rivermouths WHERE {{geostore_env}}='{{geostore_id}}' GROUP BY hyriv_id, hybas_id_5, osm_name, ord_flow, variable, dt ORDER BY hyriv_id ASC, variable ASC, dt ASC",
      "format": {
        "type": "json",
        "property": "rows",
        "parse": {
          "date": "utc:'%Y-%m-%dT%H:%M:%SZ'"
        }
      },
      "transform": [
        {
          "type": "filter",
          "expr": "datum.variable==variable"
        },
        {
          "type": "collect",
          "sort": {
            "field": "date"
          }
        }
      ]
    }
  ],
  "scales": [
    {
      "name": "xscale",
      "type": "time",
      "domain": {
        "data": "table",
        "field": "date"
      },
      "range": "width",
      "padding": 0
    },
    {
      "name": "yscale",
      "type": "linear",
      "domain": {
        "data": "table",
        "field": "avg_value"
      },
      "nice": true,
      "range": "height"
    },
    {
      "name": "colorscale",
      "type": "ordinal",
      "range": "category",
      "domain": {
        "data": "table",
        "field": "hyriv_id"
      }
    },
    {
      "name": "lineweight",
      "type": "ordinal",
      "domain": [
        1,
        2,
        3,
        4,
        5
      ],
      "range": [
        4,
        3.5,
        3,
        2.5,
        2
      ]
    }
  ],
  "axes": [
    {
      "orient": "bottom",
      "scale": "xscale",
      "labelFlush": true,
      "labelOverlap": true,
      "labelBaseline": "middle",
      "ticks": true,
      "tickSize": 5,
      "labelPadding": 8,
      "labelFontSize": 10,
      "labelFont": "Arial",
      "format": "%b %Y"
    },
    {
      "orient": "left",
      "scale": "yscale",
      "title": {
        "signal": "'Concentration (mmol/m³)'"
      },
      "labelBaseline": "middle",
      "labelAlign": "right",
      "labelPadding": 3,
      "grid": true,
      "gridOpacity": 0.2,
      "labelFontSize": 10,
      "labelFont": "Arial"
    }
  ],
  "marks": [
    {
      "type": "group",
      "from": {
        "facet": {
          "name": "series",
          "data": "table",
          "groupby": "hyriv_id"
        }
      },
      "marks": [
        {
          "type": "line",
          "from": {
            "data": "series"
          },
          "encode": {
            "enter": {
              "interpolate": {
                "value": "monotone"
              },
              "x": {
                "scale": "xscale",
                "field": "date"
              },
              "y": {
                "scale": "yscale",
                "field": "avg_value"
              },
              "stroke": {
                "scale": "colorscale",
                "field": "hyriv_id"
              },
              "strokeWidth": {
                "scale": "lineweight",
                "field": "ord_flow"
              }
            },
            "update": {
              "strokeOpacity": {
                "value": 1
              }
            },
            "hover": {
              "strokeOpacity": {
                "value": 0.5
              }
            }
          }
        },
        {
          "type": "symbol",
          "from": {
            "data": "series"
          },
          "encode": {
            "enter": {
              "x": {
                "scale": "xscale",
                "field": "date"
              },
              "y": {
                "scale": "yscale",
                "field": "avg_value"
              },
              "stroke": {
                "scale": "colorscale",
                "field": "hyriv_id"
              },
              "strokeWidth": {
                "scale": "lineweight",
                "field": "ord_flow"
              },
              "size": {
                "signal": "width/6"
              },
              "tooltip": {
                "signal": "{'River':isValid(datum.name) ? datum.name : '(name unavailable)','ID':datum.hyriv_id, 'Date':timeFormat(datum.date, '%B %Y'), 'Concentration':format(datum.avg_value, '.3f')+' mmol/m³'}"
              }
            },
            "update": {
              "fill": {
                "value": "white"
              },
              "fillOpacity": {
                "value": 0
              },
              "strokeOpacity": {
                "signal": "datum.hyriv_id == active_id ? 1.0 : 0.0"
              }
            },
            "hover": {
              "fill": {
                "scale": "colorscale",
                "field": "hyriv_id"
              },
              "fillOpacity": {
                "value": 1
              }
            }
          }
        }
      ]
    }
  ]
}
""")

In [11]:
vega_view=dict(spec)

## DEFINE SIZE
vega_view['height'] = widget_height
vega_view['width'] = widget_width

## PARAMETERIZE WIDGET
vega_view['data'][0]['url']= vega_view['data'][0]['url'].replace('{{geostore_env}}','geostore_prod')
vega_view['data'][0]['url'] = vega_view['data'][0]['url'].replace('{{geostore_id}}','4053d745654f93328e561a128aac1bba')

## ADJUST X AXIS
vega_view['axes'][0]['labelBaseline']= "top"

## FONT
vega_view['axes'][1]['titleFont'] = 'Arial'

Vega(vega_view)

# RW API

Parameterized widgets created via [widget scripts](https://github.com/resource-watch/data-team-tools/tree/master/advanced_widget_writer).  

## Nitrate
[ocn.020a.rw0 Ocean Nitrate Concentration](https://resourcewatch.org/data/explore/ocn020a-Nitrate-Concentration) ([Admin](https://resourcewatch.org/admin/data/datasets/92327c78-a473-402b-8edf-409869823216/edit))  
`92327c78-a473-402b-8edf-409869823216`  
[Dataset](https://api.resourcewatch.org/v1/dataset/92327c78-a473-402b-8edf-409869823216/), [Widgets](https://api.resourcewatch.org/v1/dataset/92327c78-a473-402b-8edf-409869823216/widget)

- OLD [Nitrate](https://api.resourcewatch.org/v1/widget/80b7addc-f6ea-4d38-808c-359e49a8b84e?env=production) `80b7addc-f6ea-4d38-808c-359e49a8b84e`  
- NEW `b5d03901-ddf2-40cd-999c-dada8a7ac415`

## Phosphate
[ocn.020b.rw0 Ocean Phosphate Concentration](https://resourcewatch.org/data/explore/ocn020b-Mole-Concentration-of-Nitrate) ([Admin](https://resourcewatch.org/admin/data/datasets/f1aa9ec7-c3b6-441c-b395-96fc796b7612/edit))  
`f1aa9ec7-c3b6-441c-b395-96fc796b7612`  
[Dataset](https://api.resourcewatch.org/v1/dataset/f1aa9ec7-c3b6-441c-b395-96fc796b7612/), [Widgets](https://api.resourcewatch.org/v1/dataset/f1aa9ec7-c3b6-441c-b395-96fc796b7612/widget)

- OLD [Phosphate](https://api.resourcewatch.org/v1/widget/d0b57543-e771-41e1-a9b6-a9487d5c3d5b?env=production) `d0b57543-e771-41e1-a9b6-a9487d5c3d5b`  
- NEW `e22ed0d7-9929-499c-8dda-49e0f9418f9a`

## Oxygen
[ocn.020c.rw0 Ocean Dissolved Oxygen Concentration](https://resourcewatch.org/data/explore/ocn020c-Dissolved-Oxygen-Concentration) ([Admin](https://resourcewatch.org/admin/data/datasets/877cdf39-5536-409c-bcba-2220e1b72796/edit))  
`877cdf39-5536-409c-bcba-2220e1b72796`  
[Dataset](https://api.resourcewatch.org/v1/dataset/877cdf39-5536-409c-bcba-2220e1b72796/), [Widgets](https://api.resourcewatch.org/v1/dataset/877cdf39-5536-409c-bcba-2220e1b72796/widget)

- OLD [Oxygen](https://api.resourcewatch.org/v1/widget/3f531725-9d1f-436f-85f8-b1494b0262c1?env=production) `3f531725-9d1f-436f-85f8-b1494b0262c1`  
- NEW `511ab9fc-9fab-4025-95a3-415586421ab7`